## From mord
These first few cells are taken directly from the ordinal regression package, with some modifications for the logistic regression with weights for quantile elicitation

In [35]:
import numpy as np
from scipy import optimize
import scipy
from sklearn import base, metrics
from sklearn.utils.validation import check_X_y
from classifiers import *

## Tests

## Quantile Elicitation

In [2]:
def compute_quantile(X_test, alpha):
    ## Known from data generation
    d = 2
    w = np.array([1/d]*d) 
    portions = np.array([3, 7, 10, 15, 19, 21]) * 8
    k = 6
    
    projection_points = X_test.dot(w) * d
    return get_class_quantiles(projection_points, portions, k, alpha)
    
def get_class_quantiles(projection_points, portions, k, alpha):
    y = np.matrix(np.zeros(projection_points.size)).T
    for i in range(0, projection_points.size):
        curr_point = projection_points[i]
        likely_class = np.argmax((curr_point - portions) < 0)
        if(likely_class == k - 1):
            y[i] = k-1
        else:
            if(likely_class > 0):
                prob_up = 1 - (portions[likely_class] - curr_point) / (portions[likely_class] - portions[likely_class - 1])
            else:
                prob_up = 1 - (portions[likely_class] - curr_point) / (portions[likely_class] - 0)
            y[i] = likely_class + 1 * (prob_up > 1 - alpha)
    return y

In [3]:
from sklearn import linear_model, metrics
import pandas as pd
import numpy as np

## Quantile to be elicited
a = 0.8

X_train = pd.read_csv('quantile_synthetic_features.csv', names=['dim1', 'dim2'])
y = pd.read_csv('quantile_synthetic_labels.csv', names=['y'])
y_train = np.array(y.iloc[:, 0]).astype(int)

X_test = pd.read_csv('quantile_synthetic_manytestfeatures.csv', names=['dim1', 'dim2'])
y_test = pd.read_csv('quantile_synthetic_manytestlabels.csv', names=['y'])
y_test = np.array(y_test.iloc[:, 0]).astype(int)

clf1 = linear_model.LogisticRegression(
    solver='lbfgs',
    multi_class='multinomial')
clf1.fit(X_train, y_train)

print('Mean Absolute Error of LogisticRegression: %s' %
      metrics.mean_absolute_error(clf1.predict(X_test), y_test))
print('0-1 Loss of LogisticRegression %s' %
      metrics.zero_one_loss(clf1.predict(X_test), y_test))
print(' ')

clf2 = LogisticAT(alpha=1.)
clf2.fit(X_train, y_train)
print('Mean Absolute Error of LogisticAT %s' %
      metrics.mean_absolute_error(clf2.predict(X_test), y_test))
print('0-1 Loss of LogisticAT %s' %
      metrics.zero_one_loss(clf2.predict(X_test), y_test))
print(' ')

clf3 = LogisticIT(alpha=1.)
clf3.fit(X_train, y_train)
print('Mean Absolute Error of LogisticIT %s' %
      metrics.mean_absolute_error(clf3.predict(X_test), y_test))
print('0-1 Loss of LogisticIT %s' %
      metrics.zero_one_loss(clf3.predict(X_test), y_test))
print(' ')

clf4 = LogisticSE(alpha=1.)
clf4.fit(X_train, y_train)
print('Mean Absolute Error of LogisticSE %s' %
      metrics.mean_absolute_error(clf4.predict(X_test), y_test))
print('0-1 Loss of LogisticSE %s' %
      metrics.zero_one_loss(clf4.predict(X_test), y_test))
print(' ')

clf5 = LogisticQuantileIT(gamma=a, alpha=1.)
clf5.fit(X_train, y_train)
print('Mean Absolute Error of LogisticQuantileIT, gamma=' + str(a) + ' %s' %
      metrics.mean_absolute_error(clf5.predict(X_test), y_test))
print('0-1 Loss of LogisticQuantileIT, gamma=' + str(a) + ' %s' %
      metrics.zero_one_loss(clf5.predict(X_test), y_test))
print(' ')

clf6 = LogisticQuantileAT(gamma=a, alpha=1.)
clf6.fit(X_train, y_train)
print('Mean Absolute Error of LogisticQuantileAT, gamma=' + str(a) + ' %s' %
      metrics.mean_absolute_error(clf6.predict(X_test), y_test))
print('0-1 Loss of LogisticQuantileAT, gamma=' + str(a) + ' %s' %
      metrics.zero_one_loss(clf6.predict(X_test), y_test))
print(' ')

clf7 = LogisticQuantileDirect(gamma=a, alpha=1.)
clf7.fit(X_train, y_train)
print('Mean Absolute Error of LogisticQuantileDirect, gamma=' + str(a) + ' %s' %
      metrics.mean_absolute_error(clf7.predict(X_test), y_test))
print('0-1 Loss of LogisticQuantileDirect, gamma=' + str(a) + ' %s' %
      metrics.zero_one_loss(clf7.predict(X_test), y_test))

Mean Absolute Error of LogisticRegression: 1.7199
0-1 Loss of LogisticRegression 0.8951
 
Mean Absolute Error of LogisticAT 1.4566
0-1 Loss of LogisticAT 0.8588
 
Mean Absolute Error of LogisticIT 1.5577
0-1 Loss of LogisticIT 0.7884
 
Mean Absolute Error of LogisticSE 1.3202
0-1 Loss of LogisticSE 0.8961
 
Mean Absolute Error of LogisticQuantileIT, gamma=0.8 1.011
0-1 Loss of LogisticQuantileIT, gamma=0.8 0.7807
 
Mean Absolute Error of LogisticQuantileAT, gamma=0.8 0.6455
0-1 Loss of LogisticQuantileAT, gamma=0.8 0.4797
 
Mean Absolute Error of LogisticQuantileDirect, gamma=0.8 1.6558
0-1 Loss of LogisticQuantileDirect, gamma=0.8 0.9645


In [27]:
X_train = pd.read_csv('quantile_synthetic_features.csv', names=['dim1', 'dim2'])
y = pd.read_csv('quantile_synthetic_labels.csv', names=['y'])
y_train = np.array(y.iloc[:, 0]).astype(int)

X_test = pd.read_csv('quantile_synthetic_testfeatures.csv', names=['dim1', 'dim2'])
y_test = pd.read_csv('quantile_synthetic_testlabels.csv', names=['y'])
y_test = np.array(y_test.iloc[:, 0]).astype(int)


ATs = []
AT_predictions = []

s = 10 # Number of quantiles
for i in range(1, s):
    a = i/s

    clf6 = LogisticQuantileAT(gamma=a, alpha=1.)
    clf6.fit(X_train, y_train)
    ATpredictions = clf6.predict(X_test)
    print('Mean Absolute Error of LogisticQuantileAT, gamma=' + str(a) + ' %s' %
          metrics.mean_absolute_error(ATpredictions, y_test))
    print('0-1 Loss of LogisticQuantileAT, gamma=' + str(a) + ' %s' %
          metrics.zero_one_loss(ATpredictions, y_test))
    ATs.append(clf6)
    AT_predictions.append(ATpredictions)

Mean Absolute Error of LogisticQuantileAT, gamma=0.1 4.2673
0-1 Loss of LogisticQuantileAT, gamma=0.1 0.9132
Mean Absolute Error of LogisticQuantileAT, gamma=0.2 3.3893
0-1 Loss of LogisticQuantileAT, gamma=0.2 0.8998
Mean Absolute Error of LogisticQuantileAT, gamma=0.3 2.8346
0-1 Loss of LogisticQuantileAT, gamma=0.3 0.8753
Mean Absolute Error of LogisticQuantileAT, gamma=0.4 2.5093
0-1 Loss of LogisticQuantileAT, gamma=0.4 0.87
Mean Absolute Error of LogisticQuantileAT, gamma=0.5 2.4259
0-1 Loss of LogisticQuantileAT, gamma=0.5 0.8729
Mean Absolute Error of LogisticQuantileAT, gamma=0.6 2.5682
0-1 Loss of LogisticQuantileAT, gamma=0.6 0.8837
Mean Absolute Error of LogisticQuantileAT, gamma=0.7 2.9424
0-1 Loss of LogisticQuantileAT, gamma=0.7 0.9017
Mean Absolute Error of LogisticQuantileAT, gamma=0.8 3.5594
0-1 Loss of LogisticQuantileAT, gamma=0.8 0.9222
Mean Absolute Error of LogisticQuantileAT, gamma=0.9 4.5461
0-1 Loss of LogisticQuantileAT, gamma=0.9 0.9312


In [28]:
AT_preds = np.zeros((10000, 10))
for i in range(1, 10):
    AT_preds[:, i] = AT_predictions[i - 1]

In [34]:
sum(scipy.stats.mode(AT_preds.T).mode[0])

46516

In [29]:
print(metrics.mean_absolute_error(scipy.stats.mode(AT_preds.T).mode[0], y_test.flatten()))
print(metrics.zero_one_loss(scipy.stats.mode(AT_preds.T).mode[0], y_test.flatten()))

4.0864
0.886


In [30]:
clf1 = linear_model.LogisticRegression(
    solver='lbfgs',
    multi_class='multinomial')
clf1.fit(X_train, y_train)

print('Mean Absolute Error of LogisticRegression: %s' %
      metrics.mean_absolute_error(clf1.predict(X_test), y_test))
print('0-1 Loss of LogisticRegression %s' %
      metrics.zero_one_loss(clf1.predict(X_test), y_test))
print(' ')

Mean Absolute Error of LogisticRegression: 2.8393
0-1 Loss of LogisticRegression 0.8298
 


## Plot Threshold Boundaries

In [ ]:
from matplotlib import pyplot as plt
def plot_clf(coef, theta):
    ## From data generation
    last_point = 170
    
    x = np.arange(0, 170, 0.5)
    y = x * (coef[0] / coef[1])
    plt.plot(x,y)
    
    for t in theta:
        t_y = x * (-coef[1] / coef[0]) + t * (2 / coef[0])
        t_y = t_y * (t_y > 0)
        plt.plot(x, t_y)

In [ ]:
plot_clf(clf5.coef_, clf5.theta_)

## Multiclass Classification

In [ ]:
k = 100
n = 10000
X_train = pd.read_csv('multiclass_train_features_k' + str(k) + 'n' + str(n) + '.csv')
y = pd.read_csv('multiclass_train_labels_k' + str(k) + 'n' + str(n) + '.csv')
y_train = np.array(y.iloc[:, 0]).astype(int)

X_test = pd.read_csv('multiclass_test_features_k' + str(k) + 'n' + str(n) + '.csv')
y = pd.read_csv('multiclass_test_labels_k' + str(k) + 'n' + str(n) + '.csv')
y_test = np.array(y.iloc[:, 0]).astype(int)

In [ ]:
ITs = []
IT_predictions = []
ATs = []
AT_predictions = []

s = 10
for i in range(1, s):
    a = i / s
    clf5 = LogisticQuantileIT(gamma=a, alpha=1.)
    clf5.fit(X_train, y_train)
    ITpredictions = clf5.predict(X_test)
    print('Mean Absolute Error of LogisticQuantileIT, gamma=' + str(a) + ' %s' %
          metrics.mean_absolute_error(ITpredictions, y_test))
    ITs.append(clf5)
    IT_predictions.append(ITpredictions)

    clf6 = LogisticQuantileAT(gamma=a, alpha=1.)
    clf6.fit(X_train, y_train)
    ATpredictions = clf6.predict(X_test)
    print('Mean Absolute Error of LogisticQuantileAT, gamma=' + str(a) + ' %s' %
          metrics.mean_absolute_error(ATpredictions, y_test))
    ATs.append(clf6)
    AT_predictions.append(ATpredictions)

#     clf7 = LogisticQuantileDirect(gamma=a, alpha=1.)
#     clf7.fit(X_train, y_train)
#     print('Mean Absolute Error of LogisticQuantileDirect, gamma=' + str(a) + ' %s' %
#           metrics.mean_absolute_error(clf7.predict(X_test), y_test))

In [ ]:
## Logistic Regression for comparison
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train.flatten())

In [ ]:
sum(clf.predict(X_test) != y_test.flatten()) / len(X_test)

In [ ]:
clf.predict(X_test)

In [ ]:
metrics.mean_absolute_error(clf.predict(X_test), y_test.flatten())

In [ ]:
np.set_printoptions(threshold=np.nan)
full = np.zeros((4999, 2))
full[:, 0] = clf.predict(X_test)
full[:,1] = y_test
print(full)

In [ ]:
## OVA SVM
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train, y_train.flatten()) 

print(sum(lin_clf.predict(X_test) != y_test.flatten()) / len(X_test))

In [ ]:
np.set_printoptions(threshold=np.nan)
full = np.zeros((4999, 2))
full[:, 0] = AT_predictions[1]
full[:,1] = y_test
print(full)

In [ ]:
sum(AT_predictions[4] != y_test.flatten()) / len(X_test)

In [ ]:
AT_preds = np.zeros((4999, 10))

In [ ]:
for i in range(1, 10):
    AT_preds[:, i] = AT_predictions[i - 1]

In [ ]:
np.sum(scipy.stats.mode(AT_preds.T).mode!= y_test.flatten()) / len(X_test)

In [ ]:
metrics.mean_absolute_error(scipy.stats.mode(AT_preds.T).mode, y_test.flatten())

In [ ]:
IT_preds = np.zeros((4999, 10))
for i in range(1, 10):
    IT_preds[:, i-1] = IT_predictions[i - 1]

In [ ]:
np.sum(scipy.stats.mode(IT_preds.T).mode!= y_test.flatten()) / len(X_test)

## Simplex Testing